In [1]:
# DFS running combat

A series refers to the sequence of subactions and relevant details needed by the process_turn function.

A full_list refers to the collection of series options an actor has to pick from.

In [2]:
import multiprocessing
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from DFS_World_States import world, world_grid_states, Weapon, Potion, Shield, Wand
from DFS_Universal_Rules import theoretical_turn_length, subaction_dict, target_distance_scores, fire_bolt, ray_of_frost, shield
from DFS_Action_Series import RuleBasedSequenceDFS, action_rules 
from DFS_Location_Series import RuleBasedLocationSequenceDFS2, location_rules
from DFS_Functions import precalc_reward_series, analyze_reward_distribution, post_loc_series_reward_calc, post_obj_reward_series_calc, post_obj_reward_series_calc2, analyze_reward_distribution_series, calc_new_reward
from DFS_Object_Series import RuleBasedObjectSequenceDFS1, object_rules
from DFS_Entity_Series import RuleBasedEntitySequenceDFS, entity_rules

In [4]:
from DFS_Entities import entity

In [5]:
stage = world(11)

stage.generate_map()

actor = entity('player character',stage,'medium')
stage.add_entity([0,0],actor)

actor.add_spell(fire_bolt)
actor.add_spell(ray_of_frost)
actor.add_spell(shield)

stage.add_item_to_inventory(actor, Weapon('dagger1', 'weapon', 1, 1, 0))
stage.add_item_to_inventory(actor, Weapon('dagger2', 'weapon', 1, 1, 0))
stage.add_item_to_inventory(actor, Weapon('greataxe', 'weapon', 1, 1, 0))
stage.add_item_to_inventory(actor, Shield('shield','shield', 1))
stage.add_item_to_inventory(actor, Wand('wand','wand', 1))



monster_one = entity('monster',stage,'small','goblin','1x1',False,6)
stage.add_entity([2,2],monster_one)

monster_two = entity('monster',stage,'medium','guard','1x1',False,6)
stage.add_entity([1,1],monster_two)

monster_three = entity('monster',stage,'large','horse','2x2',True,12)
stage.add_entity([0,-3],monster_three)


#stage.add_enemy((-2,0))
#stage.add_enemy((0,3))
#stage.add_enemy((2,2))
#stage.add_enemy((-3,-3))
#stage.add_enemy((1,-1))

stage.add_coin((3,0))
stage.add_coin((-1,-1))
stage.add_coin((1,1))




stage.add_weapon_dagger((-2,-1))
stage.add_weapon_dagger((-1,-2))
stage.add_weapon_greataxe((2,0))


In [6]:
#stage.grid2

In [7]:
# I should display a visual here of the grid and where each entity and object is located

# stage.grid2 is a inhomogeneous list of lists of lists, ints, and strings that represent the grid

# I only want to see the entities and objects, so I will create a new list of lists that only contains the entities and objects




In [8]:
action_sequence_generator = RuleBasedSequenceDFS(min_length=0,max_length=theoretical_turn_length, start=0, end=len(subaction_dict), rules=action_rules, acting_entity=actor)
action_series_full_list = action_sequence_generator.generate_sequences()
#action_series_full_list

In [9]:
reward_series_full_list = precalc_reward_series(action_series_full_list, actor)
#reward_series_full_list

In [10]:
#analyze_reward_distribution(reward_series_full_list, action_series_full_list)
action_reward_series_full_list = [[action_series_full_list[i], reward_series_full_list[i]] for i in range(len(action_series_full_list))]

analyze_reward_distribution_series(action_reward_series_full_list, actor, 99)

Total count: 2050851
High: 8
Low: -2

Reward Distribution:
% of -2s: 0.06%
% of -1s: 1.03%
% of 0s: 5.96%
% of 1s: 15.88%
% of 2s: 31.54%
% of 3s: 29.10%
% of 4s: 11.68%
% of 5s: 3.81%
% of 6s: 0.86%
% of 7s: 0.08%
% of 8s: 0.00%

Quality Threshold (99): 5.0
Total Qualifiers: 97316



In [11]:
location_series_generator = RuleBasedLocationSequenceDFS2(action_series_list=action_series_full_list,
                                                             reward_series_list=reward_series_full_list,
                                                             location_rules=location_rules,
                                                             acting_entity=actor,
                                                             target_distance_scores=target_distance_scores)
location_series_full_list = location_series_generator.generate_sequences()

Quality threshold: 5.0


TypeError: 'NoneType' object is not iterable

In [10]:
post_location_reward_list = post_loc_series_reward_calc(action_series_full_list, location_series_full_list, reward_series_full_list, actor)

In [ ]:
analyze_reward_distribution_series(post_location_reward_list, actor, 99)

In [12]:
# Object Series
object_series_generator = RuleBasedObjectSequenceDFS1(post_location_reward_list = post_location_reward_list, 
                                                     target_distance_scores = target_distance_scores,
                                                     object_rules = object_rules, 
                                                     acting_entity = actor)
object_series_full_list = object_series_generator.generate_object_sequences()

In [ ]:
post_object_reward_list = post_obj_reward_series_calc2(object_series_full_list, actor)
post_object_reward_list

In [ ]:
analyze_reward_distribution_series(post_object_reward_list, actor, 99)

In [15]:
# entity series
# in order to specify which entity is being targeted

# shortcuts:
# - currently based on the location series, if there is only one entity in a space, the entity is automatically targeted
# - only if there are multiple entities in a space, the entity series is used to determine the target

In [ ]:
entity_series_generator = RuleBasedEntitySequenceDFS(action_location_object_reward_series_full_list = post_object_reward_list,
                                                     target_distance_scores = target_distance_scores,
                                                     entity_rules = entity_rules,
                                                     acting_entity = actor)
entity_series_generator.generate_entity_sequences()

In [17]:
action_location_object_entity_reward_series_full_list = entity_series_generator.generate_entity_sequences()

In [ ]:
action_location_object_entity_reward_series_full_list

In [ ]:
action_location_object_entity_reward_series_full_list = calc_new_reward(action_location_object_entity_reward_series_full_list, actor)

In [ ]:
analyze_reward_distribution_series(action_location_object_entity_reward_series_full_list, actor, 99)

In [ ]:
# spell series


In [24]:
#post_object_reward_list = post_obj_reward_series_calc(action_series_full_list = action_series_full_list, 
#                                                      location_series_full_list = location_series_full_list, 
#                                                      object_series_full_list = object_series_full_list, 
#                                                      reward_series_full_list = reward_series_full_list, 
#                                                      acting_entity = actor)

# this one will include how much damage is being dealt
# if an effective weapon is equipped
# how many coins are picked up

In [26]:
# processing a round of combat that includes the queue of actions and possible reactions other entities take

In [27]:
# what needs to happen before I can be running combat?
# - the function to process a creature's turn needs to be done
# - the function to process everyone else's reaction needs to be operational
# - enemies need to be able to take their turn, currently they operate as a form of static entity



In [28]:
# what needs to happen before the next stage?
# - combat situations needs to be randomly generated
# - combat scenarios need to be ran millions of times